<a href="https://colab.research.google.com/github/Jarde01/C-Memory-Regions/blob/master/Pytorch_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pytorch

  ERROR: Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
  Running setup.py install for pytorch ... error
ERROR: Command "/usr/bin/python3 -u -c 'import setuptools, tokenize;__file__='"'"'/tmp/pip-install-099rr7ht/pytorch/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-is97x7_h/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-099rr7ht/pytorch/


In [0]:
# Pytorch Transformer
### From https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec

In [0]:
import numpy as np
import torch

In [0]:
import torch.nn as nn

class Embedder(nn.Module):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.embed = nn.Embedding(vocab_size, d_model)
  def forward(self, x):
    return self.embed(x)

In [0]:
class PositionalEncoder(nn.Module):
  def __init__(self, d_model, max_seq_len = 50):
    super().__init__()
    self.d_model = d_model
    
    # positional encoding matrix
    pe = torch.zeros(max_seq_len, d_model)
    for pos in range(max_seq_len):
      for i in range(0, d_model, 2):
        pe[pos, i]= math.sin(pos/10000 ** ((2*i)/d_model))
        pe[pos, i+1] = math.cos(pos/ (10000 ** ((2 *( i + 1))/d_model)))
    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)
    
  def forward(self, x):
    # making embeddings larger1
    x = x*math.sqrt(self.d_model)

    # constant to embedding
    seq_len = x.size(1)
    x = x + Variable(self.pe[:,:seq_len], requires_grad=False).cuda()
    return x

In [6]:
batch = next(iter(train_iter))
input_seq = batch.English.transpose(0,1)
input_pad = EN_TEXT.vocab.stoi['<pad>']

# Creating a mask with 0 whenever padding in input
input_msk = (input_seq != input_pad).unsqueeze(1)

NameError: ignored

In [8]:
target_seq = batch.French.transpose(0,1)
target_pad = FR_TEXT.vocab.stoi['<pad>']
target_mask = (target_seq != target_pad).unsqueeze(1)

size = target_seq.size(1)
nopeak_mask = np.triu(np.ones(1,size,size), k=1).astype('uint8')
nopeak_mask = Variable(torch.from_numpy(nopeak_mask) == 0)

target_msk = target_msk & nopeak_msk

NameError: ignored

In [14]:
nopeak_mask = np.triu(np.ones((1,10,10)),k=1).astype('uint8')
nopeak_mask = torch.from_numpy(subsequent_mask) == 0
print(nopeak_mask)


NameError: ignored

In [0]:
class MultiheadAttention(nn.Module):
  def __init__(self, heads, d_model, dropout=0.1):
    super().__init__()
    self.d_model = d_model
    self.d_k = d_model //heads
    self.h = heads
    
    self.q_linear = nn.Linear(d_model, d_model)
    self.v_linear = nn.Linear(d_model, d_model)
    self.k_linear = nn.Linear(d_model, d_model)
    self.dropout = nn.Dropout(dropout)
    self.out = nn.Linear(d_model, d_model)
    
  def forward(self, q, k, v, mask=None):
    bs = q.size(10)
    
    # linear ops and splitting onto heads
    k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
    q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
    v = self.v_linear(v).view(bs, -1, self.h, self.d_k)

    # transport --> bs * h * s1 * d_model
    k = k.transpose(1,2)
    q = q.transpose(1,2)
    v = v.transpose(1,2)
    
    # calculation of attention
    scores = attention(q,k,v, self.d_k, mask, self.dropout)
    
    # concat heads
    concat = scores.transpose(1,2).contiguous().view(bs,-1, self.d_model)
    
    output = self.out(concat)
    return output
  
  def attention(q,k,v,d_k,mask=None,dropout=None):
    print(f"k: {k.shape}")
    print(f"k.transpose(-2,-1): {k.transpose(-2,-1)}")
    scores = torch.matmul(q,k.transpose(-2,-1)) / math.sqrt(d_k)
    
    if mask is not None:
      mask = mask.unsqueeze(1)
      print(f"scores before mask: {scores}")
      scores = scores.masked_fill(mask==0, -1e9)
      
    scores = F.softmax(scores, dim=-1)
    print(f"scores after mask: {scores}")
    
    if dropout is not None:
      scores = dropout(scores)
      
    output = torch.matmul(scores, v)
    return output

In [0]:
class FeedForward(nn.Module):
  def __init__(self, d_model, d_ff=2048, dropout=0.1):
    super().__init__() 
    # We set d_ff as a default to 2048
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)
    
  def feedforward(self,x):
    x = self.dropout(F.relu(self.liniear_1(x)))
    x = self.linear_2(x)
    return x

In [0]:
class Normalize(nn.Module):
  def __init__(self, d_model, epsilon = 1e-6):
    super().__init__()
    
    self.size = d_model
    
    # learnable params for normalization calibration
    self.alpha = nn.Parameter(torch.ones(self.size))
    self.bias = nn.Parameter(torch.zeros(self.size))
    self.epsilon = epsilon
    
  def forward(self, x):
    numer = x-x.mean(dim=1, keepdim=True)
    denom = x.std(dim=-1, keepdim=True) + self.epsilon
    
    norm = self.alpha * numer / denom + self.bias
    return norm
    

In [0]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        
    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x  

In [0]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, heads, dropout=0.1):
    super().__init__()
    self.norm_1 = Norm(d_model)
    self.norm_2 = Norm(d_model)
    self.norm_3 = Norm(d_model)

    self.dropout_1 = nn.Dropout(dropout)
    self.dropout_2 = nn.Dropout(dropout)
    self.dropout_3 = nn.Dropout(dropout)

    self.attn_1 = MultiHeadAttention(heads, d_model)
    self.attn_2 = MultiHeadAttention(heads, d_model)
    self.ff = FeedForward(d_model).cuda()
      
  def forward(self, x, e_outputs, src_mask, trg_mask):
    x2 = self.norm_1(x)
    x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
    x2 = self.norm_2(x)
    x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
    src_mask))
    x2 = self.norm_3(x)
    x = x + self.dropout_3(self.ff(x2))
    return x

In [0]:
def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [0]:
class Encooder(nn.Module):
  def __init__(self, vocab_size, d_model, N, head):
    super().__init__()
    self.N = N
    self.embed = Embedder(vocab_size, d_model)
    self.pe = PositionalEncoder(d_model)
    self.layers = get_clones(EncoderLayer(d_model, heads), N)
    self.norm = Normalize(d_model)
    
  def forward(self, src, mask):
    x = self.embed(src)
    x = self.pe(x)
    for i in range(N):
      x = self.layers[i](x, mask)
    return self.norm(x)

In [0]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, d_model, N, heads):
    super().__init__()
    self.N = N
    self.embed - Embedder(vocab_size, d_model)
    self.pe = PositionalEncoder(d_model)
    self.layers = get_clones(DecoderLayer(d_model, heads), N)
    self.norm = Normalize(d_model)
    
  def forward(self, trg, e_outputs, src_mask, trg_mask):
    x = self.embed(trg)
    x = self.pe(x)
    for i in range(self.N):
      x = self.layers[i](x, e_outputs, src_mask, trg_mask)
    return self.norm(x)

In [0]:
class Transformer(nn.Module):
  def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
    super().__init__()
    self.encoder = Encoder(src_vocab, d_model, N, heads)
    self.decoder = Decoder(trg_vocab, d_model, N, heads)
    self.out = nn.Linear(d_model, trg_mask)
    
  def forward(self, src, trg, src_mask, trg_mask):
    e_outputs = self.encoder(src, src_mask)
    d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
    output = self.out(d_output)
    return output